In [409]:
#python libraty
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

#analysis the dataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

#machine learning method
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [410]:
#01:get the train and test dataset
train_df = pd.read_csv('D:/kaggleCompetition/TitanicDataSet/train.csv',header=0)  
test_df = pd.read_csv('D:/kaggleCompetition/TitanicDataSet/test.csv',header=0)

In [411]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [412]:
#drop the unnecessary columns,these columns won't be useful in analysis and prediction
train_df = train_df.drop(["PassengerId","Name","Ticket","Cabin"],axis=1)

In [413]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [414]:
#fill the mssing data
#01 age use the media fill the missing data
train_df['Age'][train_df['Age'].isnull()] = train_df['Age'].median()  
#02 Embarked use the mode fill the missing data
train_df['Embarked'][train_df['Embarked'].isnull()] = train_df['Embarked'].mode().values 

In [415]:
#one hot encoder Sex, Embarked
train_df = pd.get_dummies(train_df,columns = train_df[["Sex","Embarked"]])

In [416]:
# merge the two column SibSp and Parch get a new feature FamilySize
train_df['Family_size'] = train_df['SibSp'] + train_df['Parch'] 

In [417]:
#drop the no need column data
train_df = train_df.drop(["SibSp","Parch"],axis=1)

In [418]:
#standardization the column about Age and Fare
train_std_df = train_df[["Age","Fare"]].apply(lambda x : (x - np.min(x))/(np.max(x) - np.min(x)))
train_drop_df = train_df.drop(["Age","Fare"],axis=1)

In [419]:
#use the concat method merge two dataFrame
train_df = pd.concat([train_drop_df,train_std_df],axis=1)

In [420]:
train_df.head()

,Survived,Pclass,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Family_size,Age,Fare
0,0,3,0,1,0,0,1,1,0.271174,0.014151
1,1,1,1,0,1,0,0,1,0.472229,0.139136
2,1,3,1,0,0,0,1,0,0.321438,0.015469
3,1,1,1,0,0,0,1,1,0.434531,0.103644
4,0,3,0,1,0,0,1,0,0.434531,0.015713


In [421]:
train_df.shape

(891, 10)

In [422]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Survived       891 non-null int64
Pclass         891 non-null int64
Sex_female     891 non-null uint8
Sex_male       891 non-null uint8
Embarked_C     891 non-null uint8
Embarked_Q     891 non-null uint8
Embarked_S     891 non-null uint8
Family_size    891 non-null int64
Age            891 non-null float64
Fare           891 non-null float64
dtypes: float64(2), int64(3), uint8(5)
memory usage: 39.2 KB


In [423]:
#define the training dataSet
train_X = train_df.drop("Survived",axis=1)
train_Y = train_df["Survived"]

In [424]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [425]:
test_df = test_df.drop(["Name","Ticket","Cabin"],axis=1)

In [426]:
test_df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S


In [427]:
#fill the mssing data
#01 age use the median fill the missing data
test_df['Age'][test_df['Age'].isnull()] = test_df['Age'].median()  
#02 Fare use the median fill the missing data
test_df['Fare'][test_df['Fare'].isnull()] = test_df['Fare'].median()  

In [428]:
#one hot encoder Sex, Embarked
test_df = pd.get_dummies(test_df,columns = test_df[["Sex","Embarked"]])

In [429]:
# merge the two column SibSp and Parch get a new feature FamilySize
test_df['Family_size'] = test_df['SibSp'] + test_df['Parch'] 

In [430]:
#drop the no need column data
test_df = test_df.drop(["SibSp","Parch"],axis=1)

In [431]:
test_std_df = test_df[["Age","Fare"]].apply(lambda x : (x - np.min(x))/(np.max(x) - np.min(x)))
test_drop_df = test_df.drop(["Age","Fare"],axis=1)

In [432]:
test_df = pd.concat([test_drop_df,test_std_df],axis=1)

In [433]:
test_df.head()

,PassengerId,Pclass,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Family_size,Age,Fare
0,892,3,0,1,0,1,0,0,0.452723,0.015282
1,893,3,1,0,0,0,1,1,0.617566,0.013663
2,894,2,0,1,0,1,0,0,0.815377,0.018909
3,895,3,0,1,0,0,1,0,0.353818,0.016908
4,896,3,1,0,0,0,1,2,0.287881,0.023984


In [434]:
test_df.shape

(418, 10)

In [435]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Sex_female     418 non-null uint8
Sex_male       418 non-null uint8
Embarked_C     418 non-null uint8
Embarked_Q     418 non-null uint8
Embarked_S     418 non-null uint8
Family_size    418 non-null int64
Age            418 non-null float64
Fare           418 non-null float64
dtypes: float64(2), int64(3), uint8(5)
memory usage: 18.4 KB


In [436]:
#get the test dataSet
test_X = test_df.drop("PassengerId",axis=1).copy()

In [437]:
#use the machine learning method train our model
#01:KNN
knn_clf = KNeighborsClassifier(n_neighbors = 5)
knn_clf.fit(train_X,train_Y)
predict_Y = knn_clf.predict(test_X)
#elevate our model
knn_clf.score(train_X,train_Y)

0.85297418630751964

In [439]:
#02:LR
# Logistic Regression
params = {'C':[100,1000],'tol':[0.001,0.0001]}
lg_clf = LogisticRegression(solver='lbfgs',multi_class='multinomial')
#tune the params
log_clf = GridSearchCV(lg_clf,params,scoring = 'log_loss',refit='true',n_jobs=-1,cv=5)
lg_clf.fit(train_X,train_Y)
predict_Y = lg_clf.predict(test_X)
lg_clf.score(train_X,train_Y)

0.80022446689113358

In [440]:
#03:Random Forests
rf_clf = RandomForestClassifier(n_estimators=1000)
rf_clf.fit(train_X,train_Y)
predict_Y = rf_clf.predict(test_X)
rf_clf.score(train_X,train_Y)

0.97979797979797978

In [441]:
#04: Neural network
nn_clf = MLPClassifier(solver='lbfgs',alpha=1e-5,hidden_layer_sizes=(15,10), random_state=1)
nn_clf.fit(train_X,train_Y)
predict_Y = nn_clf.predict(test_X)
nn_clf.score(train_X,train_Y)

0.84062850729517391

In [442]:
#submit our predict result
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": predict_Y
    })
submission.to_csv('D:/kaggleCompetition/TitanicDataSet/submit_titanic_result.csv', index=False)

In [443]:
#this method used to plot the different the predict and the true
"""
plt.figure()
plt.plot(np.arange(len(predict_Y)),test_y,'go-',label='true value')
plt.plot(np.arange(len(predict_Y)),predict_Y,'ro-',label='predict value')
plt.title('score: %f'%score)
plt.legend()
plt.show()
"""

"\nplt.figure()\nplt.plot(np.arange(len(predict_Y)),test_y,'go-',label='true value')\nplt.plot(np.arange(len(predict_Y)),predict_Y,'ro-',label='predict value')\nplt.title('score: %f'%score)\nplt.legend()\nplt.show()\n"